In [1]:
import os
%pwd

'd:\\text_summerisation\\textsummerizer\\research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\text_summerisation\\textsummerizer'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    weight_decay: float
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int
    report_to : str


In [7]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [12]:
class ConfigurationManager:
    def __init__(self,
                config_filepath=CONFIG_FILE_PATH,
                params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_training_config(self)->ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.TrainingArguments
        create_directories([config.root_dir])

        model_trainer_config=ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            per_device_eval_batch_size = params.per_device_eval_batch_size,
            weight_decay = params.weight_decay,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps,
            report_to=params.report_to
        )
        return model_trainer_config

    


In [9]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from peft import get_peft_model, LoraConfig, TaskType
import torch
from datasets import load_from_disk

d:\text_summerisation\textsummerizer\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-03-01 22:39:56,435: INFO: config: PyTorch version 2.6.0 available.]


In [19]:
pip install bitsandbytes --no-cache-dir

Note: you may need to restart the kernel to use updated packages.


In [20]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [ ]:
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config=config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)

        model_Falcons = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt,
        torch_dtype=torch.float16,quantization_config={"load_in_4bit": True}).to(device)

        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_Falcons)

        # Configure LoRA
        lora_config = LoraConfig(
            task_type=TaskType.SEQ_2_SEQ_LM,  # For text summarization models
            r=16, lora_alpha=32, lora_dropout=0.05,
            bias="none"
        )

        # Apply LoRA to the model
        model = get_peft_model(model, lora_config)
        model.print_trainable_parameters()

        #loading the data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01,
            evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16,
             report_to= "none"
        ) 
        trainer = Trainer(model=model_Falcons, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"],
                  eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        ## Save model
        model_Falcons.save_pretrained(os.path.join(self.config.root_dir,"Falcons-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [14]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.48.1
Uninstalling transformers-4.48.1:
  Successfully uninstalled transformers-4.48.1
Found existing installation: accelerate 1.3.0
Uninstalling accelerate-1.3.0:
  Successfully uninstalled accelerate-1.3.0
  Using cached transformers-4.48.1-py3-none-any.whl.metadata (44 kB)
  Using cached accelerate-1.3.0-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.48.1-py3-none-any.whl (9.7 MB)
Using cached accelerate-1.3.0-py3-none-any.whl (336 kB)


In [17]:
config=ConfigurationManager()
model_trainer_config=config.get_model_training_config()
model_trainer=ModelTrainer(model_trainer_config)
model_trainer.train()

[2025-03-01 22:49:41,170: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-01 22:49:41,177: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-01 22:49:41,180: INFO: common: created directory at: artifacts]
[2025-03-01 22:49:41,181: INFO: common: created directory at: artifacts/model_trainer]


CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend